<a href="https://colab.research.google.com/github/Johnny880724/AFSA_material/blob/main/fish_on_real_crystal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Mount Google Drive to access your files
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Import necessary libraries
import numpy as np

# Step 3: Define the path to your .txt file
# You can find the path after mounting your Google Drive.
# Make sure to adjust the path based on where your file is located in Drive.
file_path = '/content/drive/MyDrive/crystal data/5A_parsed_file.txt'

# Step 4: Load the .txt file into a NumPy array
# Assuming the file is space-separated, but adjust delimiter if needed
matrix = np.loadtxt(file_path)

# Step 5: Display the matrix

print("Matrix Loaded from File:")
print(matrix[1])


Mounted at /content/drive
Matrix Loaded from File:
[-1.532579e+02  4.961000e+00  6.140000e-16 -3.787000e+00  0.000000e+00
  1.864000e+01  4.770000e-15  0.000000e+00  0.000000e+00  6.941000e+00]


MultiGaussian fitting

In [4]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

def fit_gaussian_process(input_array):
    """
    Fit a Gaussian Process model to the provided n-dimensional array.

    Args:
    - input_array (np.ndarray): The n-dimensional array where the first row contains energy (z-values)
                                and the subsequent rows represent the coordinates in n-dimensional space.

    Returns:
    - gp (GaussianProcessRegressor): The trained Gaussian Process model.
    """
    # Separate the energy (first row) and location data (other rows)
    energy_values = input_array[0, :]  # Energy values (z)
    location_data = input_array[1:, :].T  # Location data (x, y, ...)

    # Define the kernel: we use a product of constant kernel and RBF kernel (Gaussian kernel)
    kernel = C(1.0, (1e-4, 1e1)) * RBF(1.0, (1e-4, 1e1))  # Constant * RBF kernel

    # Initialize the GaussianProcessRegressor with the kernel
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

    # Fit the GP model to the data
    gp.fit(location_data, energy_values)

    return gp

def predict_energy(gp, location):
    """
    Predict the energy at a given location using the fitted Gaussian Process model.

    Args:
    - gp (GaussianProcessRegressor): The trained Gaussian Process model.
    - location (np.ndarray): The location for which to predict the energy.

    Returns:
    - energy (float): The predicted energy at the given location.
    """
    # Ensure the location is in the right shape (1 x n_dim)
    location = np.array(location).reshape(1, -1)

    # Use the Gaussian Process model to predict the energy at the specified location
    energy, _ = gp.predict(location, return_std=True)

    return energy[0]  # Return the predicted energy value

# Example Usage:
# Input: An example n-dimensional array where the first row is energy, and others are coordinates.
input_array = np.array([
    [1.0, 2.0, 3.0, 4.0],  # Energy (z-values)
    [0.0, 1.0, 2.0, 3.0],  # Location 1 (x)
    [1.0, 1.5, 2.0, 2.5],  # Location 2 (y)
])

# Fit the Gaussian Process model to the input data
gp_model = fit_gaussian_process(matrix.T)

# Predict the energy at a new location (e.g., at location [2.5, 2.0])
new_location = [0,1,0,0,0,1,1,0,1]

predicted_energy = predict_energy(gp_model, new_location)

# Output the predicted energy
print(f"Predicted energy at location {new_location}: {predicted_energy}")


Predicted energy at location [0, 1, 0, 0, 0, 1, 1, 0, 1]: -0.00044713430777307094


/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


Fish sworm


In [5]:
import numpy as np

class ArtificialFishSwarm:
    def __init__(self, objective_function, n_dim, population_size=50, max_iter=100, step_size=0.5, visual_range=1.0, max_steps=10):
        """
        Initialize the Artificial Fish Swarm algorithm.

        :param objective_function: The objective function to minimize (or maximize).
        :param n_dim: The number of dimensions in the search space.
        :param population_size: The number of fish (individuals) in the swarm.
        :param max_iter: The maximum number of iterations to run the algorithm.
        :param step_size: The step size for fish movement.
        :param visual_range: The visual range of each fish to explore.
        :param max_steps: The maximum number of steps a fish can take in one iteration.
        """
        self.objective_function = objective_function
        self.n_dim = n_dim
        self.population_size = population_size
        self.max_iter = max_iter
        self.step_size = step_size
        self.visual_range = visual_range
        self.max_steps = max_steps

        # Initialize the fish population
        self.population = np.random.uniform(-10, 10, (self.population_size, self.n_dim))  # Adjust bounds if necessary
        self.fitness = np.apply_along_axis(self.objective_function, 1, self.population)

        # Initialize the best solution
        self.best_fish = self.population[np.argmin(self.fitness)]
        self.best_fitness = np.min(self.fitness)

    def move_fish(self, fish, target):
        """
        Move a fish towards a target fish.

        :param fish: The current fish position.
        :param target: The target fish position.
        :return: The new position of the fish.
        """
        direction = target - fish
        distance = np.linalg.norm(direction)

        if distance > 0:
            direction = direction / distance  # Normalize direction

        # Move the fish in the direction of the target
        new_position = fish + self.step_size * direction

        # Ensure new position is within bounds
        return np.clip(new_position, -10, 10)

    def search_neighborhood(self, fish):
        """
        Search the neighborhood of the fish and move to the best food source.

        :param fish: The current fish position.
        :return: The best position found in the neighborhood.
        """
        best_fish_in_neighborhood = fish
        best_fitness_in_neighborhood = self.objective_function(fish)

        # Randomly search for a better position within visual range
        for _ in range(self.max_steps):
            move_direction = np.random.uniform(-self.visual_range, self.visual_range, self.n_dim)
            new_fish = fish + move_direction

            # Ensure new fish position is within bounds
            new_fish = np.clip(new_fish, -10, 10)

            new_fitness = self.objective_function(new_fish)

            # If the new position is better, update the best fish
            if new_fitness < best_fitness_in_neighborhood:
                best_fish_in_neighborhood = new_fish
                best_fitness_in_neighborhood = new_fitness

        return best_fish_in_neighborhood

    def run(self):
        """
        Run the Artificial Fish Swarm algorithm.

        :return: The best solution found and its fitness value.
        """
        for iteration in range(self.max_iter):
            for i in range(self.population_size):
                fish = self.population[i]

                # Search the neighborhood of the fish
                best_fish_in_neighborhood = self.search_neighborhood(fish)

                # Move towards the best fish in the neighborhood
                new_fish = self.move_fish(fish, best_fish_in_neighborhood)

                # Evaluate fitness of the new fish
                new_fitness = self.objective_function(new_fish)

                # If the new position is better, update the fish and the best solution
                if new_fitness < self.fitness[i]:
                    self.population[i] = new_fish
                    self.fitness[i] = new_fitness

                    # Update global best solution
                    if new_fitness < self.best_fitness:
                        self.best_fish = new_fish
                        self.best_fitness = new_fitness

            print(f"Iteration {iteration+1}: Best Fitness = {self.best_fitness}")

        return self.best_fish, self.best_fitness

# Example: Define a simple objective function (e.g., Rastrigin function)
def objective_function(x):
    """
    A sample objective function for optimization (Rastrigin function).
    :param x: The input vector (N-dimensional).
    :return: The fitness value.
    """
    return 10 * len(x) + np.sum(x**2 - 10 * np.cos(2 * np.pi * x))

# Example usage
n_dim = 10  # N-dimensional search space
population_size = 50
max_iter = 100

# Create and run the Artificial Fish Swarm Algorithm
afsa = ArtificialFishSwarm(gp_model, n_dim, population_size, max_iter)
best_solution, best_fitness = afsa.run()

print("\nBest Solution:", best_solution)
print("Best Fitness:", best_fitness)


TypeError: 'GaussianProcessRegressor' object is not callable